<img style="float: left;padding: 1em" src="https://raw.githubusercontent.com/gw-odw/odw-2022/main/Tutorials/logo.png">  

# **gw-odw22** | data challenge<br><br>Challenge 4b

*coder*: Barone Francesco Pio  |   git@[baronefr](http://github.com/baronefr)
<br>MSc in *Physics of Data*, University of Padua, AY 2021/22


In this notebook I review the posteriors of the gravitational waves I found in the previous notebook analysis:

| H1 time | H1 SNR | H1 mass | L1 time | L1 SNR | L1 mass |
| :-: | :-: | :-: | :-: | :-: | :-: |
|1204.71| 39.65 | 22 | 1204.74 | 52.58 | 22 |
|1638.17| 32.04 | 17 | 1638.18 | 38.06 | 17 |
|2483.96| 26.64 | 10 | 2483.96 | 29.61 | 10 |
|2995.34| 10.62 | 32 | 2995.34 |  8.83 | 32 |
|3319.26| 13.7  | 36 | 3319.25 | 13.94 | 35 |

In [1]:
# candidates identified in previous notebook
dataset = [ {'id':'gwc1', 't':1204.71, 'm':22 },
            {'id':'gwc2', 't':1638.17, 'm':17 },
            {'id':'gwc3', 't':2483.96, 'm':10 },
            {'id':'gwc4', 't':2995.34, 'm':32 },
            {'id':'gwc5', 't':3319.26, 'm':35 } ]
print(len(dataset), 'gravitational waves to test')

5 gravitational waves to test


In [2]:
# add chirp mass as info
def chirp_mass(m1, m2 = None):
    if m2 is None: m2 = m1
    return (((m1*m2)**3)/(m1+m2))**(1/5)

dataset = [ {**dd, 'mc': round( chirp_mass(dd['m']), 3) } for dd in dataset ]

### requirements

In [3]:
%matplotlib inline
from __future__ import division, print_function
import numpy as np
import matplotlib.pyplot as plt

import bilby
from bilby.core.prior import Uniform, PowerLaw
from bilby.core.prior import Constraint, Cosine, Sine
from bilby.gw.conversion import convert_to_lal_binary_black_hole_parameters, generate_all_bbh_parameters

from gwpy.timeseries import TimeSeries

In [4]:
# I/O
file_name = 'challenge3.gwf'

result_dir = './ch4_posteriors/'
bilby_dir = './ch4_bilby/'

In [5]:
# define the function to run the analysis

def bayesian_analysis(entry, do_plot=False):
    global file_name, bilby_dir
    
    this_id = entry['id']
    time_of_event = entry['t']
    chirp_mass = entry['mc']
    print('++++++ processing GW', this_id, '++++++')
    
    
    # setup empty interferometers
    H1 = bilby.gw.detector.get_empty_interferometer("H1")
    L1 = bilby.gw.detector.get_empty_interferometer("L1")
    
    ## I/O ##
    
    # signal
    post_trigger_duration = 2
    duration = 4
    analysis_start = time_of_event + post_trigger_duration - duration
    print('load data @ t:', analysis_start, analysis_start + duration)
    
    H1_analysis_data = TimeSeries.read(file_name, 'H1:CHALLENGE3', analysis_start, analysis_start + duration)
    L1_analysis_data = TimeSeries.read(file_name, 'L1:CHALLENGE3', analysis_start, analysis_start + duration)
    H1.set_strain_data_from_gwpy_timeseries(H1_analysis_data)
    L1.set_strain_data_from_gwpy_timeseries(L1_analysis_data)
    
    # psd
    psd_duration = duration * 32
    psd_start_time = analysis_start - psd_duration
    print('load psd @ t:', psd_start_time, psd_start_time + psd_duration)

    H1_psd_data = TimeSeries.read(file_name, 'H1:CHALLENGE3', psd_start_time, psd_start_time + psd_duration)
    L1_psd_data = TimeSeries.read(file_name, 'L1:CHALLENGE3', psd_start_time, psd_start_time + psd_duration)
    
    psd_alpha = 2 * H1.strain_data.roll_off / duration
    H1_psd = H1_psd_data.psd(fftlength=duration, overlap=0, window=("tukey", psd_alpha), method="median")
    L1_psd = L1_psd_data.psd(fftlength=duration, overlap=0, window=("tukey", psd_alpha), method="median")
    
    H1.power_spectral_density = bilby.gw.detector.PowerSpectralDensity(
                frequency_array=H1_psd.frequencies.value, psd_array=H1_psd.value)
    L1.power_spectral_density = bilby.gw.detector.PowerSpectralDensity(
                frequency_array=H1_psd.frequencies.value, psd_array=L1_psd.value)
    
    H1.maximum_frequency = 1024  # avoid artifacts
    L1.maximum_frequency = 1024
    
    if do_plot:
        # plot psd, just to check
        fig, ax = plt.subplots()
        idxs = H1.strain_data.frequency_mask
        ax.loglog(H1.strain_data.frequency_array[idxs],
              np.abs(H1.strain_data.frequency_domain_strain[idxs]))
        ax.loglog(H1.power_spectral_density.frequency_array[idxs],
              H1.power_spectral_density.asd_array[idxs])
        ax.set_xlabel("Frequency [Hz]")
        ax.set_ylabel("Strain [strain/$\sqrt{Hz}$]")
        plt.show()
    
    
    ## SET PRIOR ##
    prior = bilby.core.prior.PriorDict()
    
    prior['mass_ratio'] = Uniform(minimum=0.99, maximum=1.00, name='mass_ratio', latex_label='$q$', unit=None, boundary=None)
    prior['chirp_mass'] = Uniform(minimum=int(chirp_mass)-2, maximum=int(chirp_mass)+2, name='chirp_mass', latex_label='$M_c$', unit='$M_{\\odot}$', boundary=None)
    prior['luminosity_distance'] = Uniform(minimum=100.0, maximum=5000.0, name='luminosity_distance', latex_label='$d_L$', boundary=None)
    prior['dec'] = Cosine(minimum=-np.pi/2, maximum=np.pi/2, name='dec', latex_label='$\\mathrm{DEC}$', unit=None, boundary=None)
    prior['ra'] = Uniform(minimum=0, maximum=2*np.pi, name='ra', latex_label='$\\mathrm{RA}$', unit=None, boundary='periodic')
    prior['theta_jn'] = Sine(minimum=0, maximum=np.pi, name='theta_jn', latex_label='$\\theta_{JN}$', unit=None, boundary=None)
    prior['psi'] = Uniform(minimum=0, maximum=np.pi, name='psi', latex_label='$\\psi$', unit=None, boundary='periodic')
    prior['a_1'] = Uniform(minimum=0, maximum=0.99, name='a_1', latex_label='$a_1$', unit=None, boundary=None)
    prior['a_2'] = Uniform(minimum=0, maximum=0.99, name='a_2', latex_label='$a_2$', unit=None, boundary=None)
    prior['tilt_1'] = Sine(minimum=0, maximum=np.pi, name='tilt_1', latex_label='$\\theta_1$', unit=None, boundary=None)
    prior['tilt_2'] = Sine(minimum=0, maximum=np.pi, name='tilt_2', latex_label='$\\theta_2$', unit=None, boundary=None)
    prior['phi_12'] = Uniform(minimum=0, maximum=2*np.pi, name='phi_12', latex_label='$\\Delta\\phi$', unit=None, boundary='periodic')
    prior['phi_jl'] = Uniform(minimum=0, maximum=2*np.pi, name='phi_jl', latex_label='$\\phi_{JL}$', unit=None, boundary='periodic')
    prior['time_jitter'] = Uniform(minimum=-0.000244140625, maximum=0.000244140625, name='time_jitter', latex_label='$t_j$', unit=None, boundary='periodic')
    prior['phase'] = Uniform(name='phase', minimum=0, maximum=2*np.pi, boundary='periodic')
    
    #prior['geocent_time'] = Uniform(name="geocent_time", minimum=time_of_event-0.1, maximum=time_of_event+0.1)
    prior['geocent_time'] = time_of_event - 2  # correct for padding 2sx
    
    
    # set likelihood & generator
    interferometers = [H1, L1]
    
    waveform_arguments = dict(waveform_approximant='IMRPhenomPv2',
                              reference_frequency=100., catch_waveform_errors=True)

    waveform_generator = bilby.gw.WaveformGenerator(
                frequency_domain_source_model=bilby.gw.source.lal_binary_black_hole,
                waveform_arguments=waveform_arguments,
                parameter_conversion=convert_to_lal_binary_black_hole_parameters)

    likelihood = bilby.gw.likelihood.GravitationalWaveTransient(
                interferometers, waveform_generator, priors=prior)
    
    # run bilby
    result = bilby.run_sampler(
        likelihood, prior, sampler='dynesty', outdir=bilby_dir, label=this_id,
        conversion_function=bilby.gw.conversion.generate_all_bbh_parameters,
        sample="rwalk", nact=1, walks=1, nlive=500, dlogz=0.2
        )
    
    return result



def unwrap_posterior(res, entry):
    global result_dir
    
    # a log string to print in file
    res_log = " > {} with t = {}, mass = {}, mc = {}\n".format(entry['id'], entry['t'], entry['m'], entry['mc']) 
    
    quant = ['H1_optimal_snr','L1_optimal_snr','mass_1','mass_2', 'chirp_mass']
    for qq in quant:
        tmp = res.posterior[qq].values
        lb = np.quantile(tmp, 0.05)
        ub = np.quantile(tmp, 0.95)
        median = np.quantile(tmp, 0.5)
        
        log = "{} = {}, 90% C.I. = [{}, {}]".format(qq, median, lb, ub)
        res_log = res_log + log + '\n'
    
    print(res_log)
    res_log = res_log + '\n > run info\n'
    res_log = res_log + "ln Bayes factor = {} +/- {}\n".format( res.log_bayes_factor, res.log_evidence_err)
    res_log = res_log + 'nlive =' + str( res.sampler_kwargs["nlive"] ) + '\n'
    res_log = res_log + 'dlogz =' + str( res.sampler_kwargs["dlogz"] ) + '\n'
    res_log = res_log + '\n>> Priors\n' + str(res.priors)
    
    plot_file = "{}{}_mass.png".format(result_dir, entry['id'])
    parameters = dict(mass_1=entry['m'], mass_2=entry['m'])
    res.plot_corner(parameters, filename=plot_file)
    print(' +++++++ ')
    return res_log

In [6]:
for dd in dataset:
    res = bayesian_analysis(dd)
    log = unwrap_posterior(res, dd)
    log_file = "{}{}_log.txt".format(result_dir, dd['id'])
    with open(log_file, "w") as text_file:
        text_file.write(log)

++++++ processing GW gwc1 ++++++
load data @ t: 1202.71 1206.71
load psd @ t: 1074.71 1202.71


11:26 bilby INFO    : Waveform generator initiated with
  frequency_domain_source_model: bilby.gw.source.lal_binary_black_hole
  time_domain_source_model: None
  parameter_conversion: bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters
11:26 bilby INFO    : Running for label 'gwc1', output will be saved to './ch4_bilby/'
11:26 bilby INFO    : Using lal version 7.1.7
11:26 bilby INFO    : Using lal git version Branch: None;Tag: lalsuite-v7.5;Id: bd2ad8ca448e9761e3010e519a656789332ea28a;;Builder: Unknown User <>;Repository status: UNCLEAN: Modified working tree
11:26 bilby INFO    : Using lalsimulation version 3.1.2
11:26 bilby INFO    : Using lalsimulation git version Branch: None;Tag: lalsuite-v7.5;Id: bd2ad8ca448e9761e3010e519a656789332ea28a;;Builder: Unknown User <>;Repository status: UNCLEAN: Modified working tree
11:26 bilby INFO    : Search parameters:
11:26 bilby INFO    :   mass_ratio = Uniform(minimum=0.99, maximum=1.0, name='mass_ratio', latex_label='$q$', unit=Non

0it [00:00, ?it/s]

11:26 bilby INFO    : Using sampler Dynesty with kwargs {'bound': 'multi', 'sample': 'rwalk', 'verbose': True, 'periodic': None, 'reflective': None, 'check_point_delta_t': 1800, 'nlive': 500, 'first_update': None, 'walks': 1, 'npdim': None, 'rstate': None, 'queue_size': 1, 'pool': None, 'use_pool': None, 'live_points': None, 'logl_args': None, 'logl_kwargs': None, 'ptform_args': None, 'ptform_kwargs': None, 'enlarge': 1.5, 'bootstrap': None, 'vol_dec': 0.5, 'vol_check': 8.0, 'facc': 0.2, 'slices': 5, 'update_interval': 300, 'print_func': <bound method Dynesty._print_func of <bilby.core.sampler.dynesty.Dynesty object at 0x7f0cd7cf5430>>, 'dlogz': 0.2, 'maxiter': None, 'maxcall': None, 'logl_max': inf, 'add_live': True, 'print_progress': True, 'save_bounds': False, 'n_effective': None, 'maxmcmc': 5000, 'nact': 1, 'print_method': 'tqdm'}
11:26 bilby INFO    : Checkpoint every check_point_delta_t = 600s
11:26 bilby INFO    : Using dynesty version 1.0.1
11:26 bilby INFO    : Using the bilby

11:27 bilby INFO    : Sampling time: 0:00:53.079915
11:27 bilby INFO    : Generating sky frame parameters.


...

  0%|          | 0/2855 [00:00<?, ?it/s]

11:33 bilby INFO    : Computing SNRs for every sample.


  0%|          | 0/2855 [00:00<?, ?it/s]

11:33 bilby INFO    : Summary of results:
nsamples: 2855
ln_noise_evidence: -8650.247
ln_evidence: -8652.007 +/-  0.084
ln_bayes_factor: -1.760 +/-  0.084



 > gwc5 with t = 3319.26, mass = 35, mc = 30.469
H1_optimal_snr = 0.8190834298677511, 90% C.I. = [0.22690847454249294, 1.6937127837140948]
L1_optimal_snr = 0.914135948725878, 90% C.I. = [0.25251736180559603, 1.8644273866729468]
mass_1 = 34.409109000225875, 90% C.I. = [32.44917775664319, 36.6000665765378]
mass_2 = 34.2377691716507, 90% C.I. = [32.2897104582885, 36.426928016747304]
chirp_mass = 29.874038370387016, 90% C.I. = [28.171030954008263, 31.773948577071458]

 +++++++ 


Summary of results:

| time (H1) | SNR (H1 & L1) | matched filtering mass | posterior mass |
| :-: | :-: | :-: | :-: |
|1204.71| 39.65, 52.58 | 22 | $$21.8^{+1.61}_{-1.57}$$ |
|1638.17| 32.04, 38.06 | 17 | $$15.97^{+1.57}_{-1.41}$$ |
|2483.96| 26.64, 29.61 | 10 | $$8.78^{+1.68}_{-1.20}$$ |
|2995.34| 10.62,  8.83 | 32 | $$30.75^{+1.62}_{-1.39}$$ |
|3319.26| 13.7,  13.94 | 36 | $$34.41^{+1.65}_{-1.53}$$ |

End of file.